
# Sample for KFServing SDK

KFServing SDK 샘플입니다. 

interenceService를 만들고 가져오고 canary를 통해 새버전의 점진적 배포를 하고 새 버전으로 승격시키고 삭제하는 예제가 있는 노트북입니다. 


In [2]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements

infereceService를 배포할 네임스페이스를 정합니다. 
아레는 default namespace를 지정합니다. 

In [3]:
namespace = utils.get_default_target_namespace()


# Define InferenceService

먼저 default endpoint spec을 정의합니다. 그리고 inferenceservice의 기본적인 spec을 정의합니다.  


In [4]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='pvc://workspace-lecture-tf/saved_model',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))))
    
isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='mnist-server', namespace=namespace),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))


# Create InferenceService

inferenceSerivce를 만들기 위해 KFServingClient를 call합니다.


In [5]:
KFServing = KFServingClient()

In [ ]:
KFServing.create(isvc)

# Check the InferenceService

In [13]:
KFServing.get('mnist-server', namespace=namespace, watch=True)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
flower-sample        True                   100                 http://flower-sample.koock.example.com/v1/model...


# request



# Add Canary to InferenceService

먼저 canary endpoint spec을 정의하고 트레픽의 10를 canary를 통해 롤아웃합니다. 


In [6]:
canary_endpoint_spec = V1alpha2EndpointSpec(
                         predictor=V1alpha2PredictorSpec(
                           tensorflow=V1alpha2TensorflowSpec(
                             storage_uri='pvc://workspace-lecture-tf/saved_model_v2',
                             resources=V1ResourceRequirements(
                                 requests={'cpu':'100m','memory':'1Gi'},
                                 limits={'cpu':'100m', 'memory':'1Gi'}))))

KFServing.rollout_canary('mnist-server', canary=canary_endpoint_spec, percent=10,
                         namespace=namespace, watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
mnist-server         False                                                                                        
mnist-server         False                                                                                        
mnist-server         True       90              10              http://mnist-server.koock.example.com/v1/models...


# Rollout more traffic to canary of the InferenceService
트래픽 비율을 카나리아 버전으로 50 %로 롤아웃합니다.

In [7]:
KFServing.rollout_canary('mnist-server', percent=50, namespace=namespace,
                         watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC  CANARY_TRAFFIC URL                                               
mnist-server         True                    50              50 http://mnist-server.koock.example.com/v1/models...


# Promote Canary to Default

In [9]:
KFServing.promote('mnist-server', namespace=namespace, watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
mnist-server         Unknown                                                                                      
mnist-server         True       100                             http://mnist-server.koock.example.com/v1/models...


In [14]:
KFServing.delete('flower-sample', namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'flower-sample',
  'group': 'serving.kubeflow.org',
  'kind': 'inferenceservices',
  'uid': '143e0848-aa87-4218-a25a-4f56d48bd4be'}}